## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various binary classifiers

In [13]:
# Exploratory data analysis and visualization

In [18]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42
k_fold_splits = 2

np.random.seed(global_random_state)

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

class_weights: { 0: .1, 1: .9}

fh = logging.FileHandler('log_sklearn_weighted.txt')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
logger.addHandler(fh)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.critical("Test")



2017-09-24 10:06:00,515 - CRITICAL - Test
2017-09-24 10:06:00,515 - CRITICAL - Test
2017-09-24 10:06:00,515 - CRITICAL - Test
2017-09-24 10:06:00,515 - CRITICAL - Test
2017-09-24 10:06:00,515 - CRITICAL - Test
2017-09-24 10:06:00,515 - CRITICAL - Test
2017-09-24 10:06:00,515 - CRITICAL - Test


In [15]:
# Setup imports

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import cross_val_predict
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from collections import Counter

global_random_state = 42

with open('data.classification.nonsampled.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (X, y) = pickle.load(f)


MemoryError: 

In [ ]:
skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True, random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = DecisionTreeClassifier(random_state=global_random_state,class_weight=class_weights)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='weighted', sample_weight=None)
    logger.info("Computed roc_auc score of: {}".format(auc))
    logger.info(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score is: {}".format(roc_auc_avg))


# Note: Unfortunately it's not directly comparable to ROC_AUC calculated in MoleculeNet at: https://arxiv.org/pdf/1703.00564.pdf 
# This is because MoleculeNet looks at a different metric (roc_auc) and also a different task (multiclass prediction across 128 bioassays simultaneously vs binary classification here)

In [ ]:
# Does an MLP classifier help?

from sklearn.neural_network import MLPClassifier

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = MLPClassifier(random_state=global_random_state,class_weight=class_weights)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='weighted', sample_weight=None)
    logger.info("Computed roc_auc score of: {}".format(auc))
    logger.info(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score is: {}".format(roc_auc_avg))



In [ ]:
# Let's try using a Random forest

skf = StratifiedKFold(n_splits=k_fold_splits,shuffle=True,random_state=global_random_state)

roc_auc_avg = 0

for train_index, test_index in skf.split(X,y) :

    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    classifier = RandomForestClassifier(n_estimators=100, class_weight=class_weights, random_state=global_random_state, n_jobs=-1)
    classifier.fit(X_train,y_train)
    y_pred = classifier.predict(X_test)
    auc = roc_auc_score(y_test, y_pred, average='weighted', sample_weight=None)
    logger.info("Computed roc_auc score of: {}".format(auc))
    logger.info(classification_report(y_test, y_pred))
    roc_auc_avg = roc_auc_avg + auc
    
roc_auc_avg = roc_auc_avg / k_fold_splits
logger.info("Average roc_auc score is: {}".format(roc_auc_avg))